In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import Adam, SGD
import torchvision
import torchvision.transforms as transforms

import sys, os, math
import argparse

In [2]:
lr=0.01
data='cifar10'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar10_vgg_ReLU.pth'
resume = False

In [3]:
if data == 'cifar10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
    testset = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
if model == 'vgg':
    from models.vgg import VGG_ReLU
    net = nn.DataParallel(VGG_ReLU('VGG16', nclass, img_width=img_width).cuda())
    
net

DataParallel(
  (module): VGG_ReLU(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, c

In [5]:
if resume:
    print(f'==> Resuming from {model_out}')
    net.load_state_dict(torch.load(model_out))

In [6]:
cudnn.benchmark = True

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
def train(epoch):
    print('Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs, _ = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        pred = torch.max(outputs, dim=1)[1]
        correct += torch.sum(pred.eq(targets)).item()
        total += targets.numel()
    print(f'[TRAIN] Acc: {100.*correct/total:.3f}')

In [9]:
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs, _ = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'[TEST] Acc: {100.*correct/total:.3f}')

    # Save checkpoint after each epoch
    torch.save(net.state_dict(), model_out)

In [10]:
if data == 'cifar10':
    epochs = [30, 20, 10]

In [11]:
count = 0

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 20.012
[TEST] Acc: 24.090
Epoch: 1
[TRAIN] Acc: 38.944
[TEST] Acc: 37.950
Epoch: 2
[TRAIN] Acc: 54.462
[TEST] Acc: 57.500
Epoch: 3
[TRAIN] Acc: 65.260
[TEST] Acc: 54.940
Epoch: 4
[TRAIN] Acc: 71.032
[TEST] Acc: 65.520
Epoch: 5
[TRAIN] Acc: 75.498
[TEST] Acc: 71.950
Epoch: 6
[TRAIN] Acc: 78.624
[TEST] Acc: 73.190
Epoch: 7
[TRAIN] Acc: 80.834
[TEST] Acc: 77.210
Epoch: 8
[TRAIN] Acc: 82.438
[TEST] Acc: 80.200
Epoch: 9
[TRAIN] Acc: 83.414
[TEST] Acc: 80.920
Epoch: 10
[TRAIN] Acc: 84.750
[TEST] Acc: 78.530
Epoch: 11
[TRAIN] Acc: 85.828
[TEST] Acc: 82.660
Epoch: 12
[TRAIN] Acc: 86.584
[TEST] Acc: 79.030
Epoch: 13
[TRAIN] Acc: 87.368
[TEST] Acc: 84.250
Epoch: 14
[TRAIN] Acc: 87.934
[TEST] Acc: 84.900
Epoch: 15
[TRAIN] Acc: 88.556
[TEST] Acc: 84.230
Epoch: 16
[TRAIN] Acc: 89.064
[TEST] Acc: 83.410
Epoch: 17
[TRAIN] Acc: 89.668
[TEST] Acc: 85.860
Epoch: 18
[TRAIN] Acc: 90.244
[TEST] Acc: 87.040
Epoch: 19
[TRAIN] Acc: 90.504
[TEST] Acc: 87.610
Epoch: 20
[TRAIN] Acc: 90.956
